In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def missing_describe(data):
    """
    Returns a pandas dataframe containing information about missing values in columns. 
        PARAMS:
            data (pd.DataFrame): pandas dataframe to look into
        RETURNS:
            missing (): contains number, count, dtype and percentage of missing values in each column    
    """
    # number of missing values in each column
    num_missing = data.isnull().sum()
    # percentage of missing values in each column
    pct_missing = num_missing/data.size
    # concat info into one dataframe and sorted by num_missing in descending order
    missing = pd.concat([data.dtypes, num_missing, pct_missing], 
                        axis=1,
                        keys = ['dtype', 'missing_count', 'missing_percent']
                       ).sort_values('missing_count', ascending=False)
    return missing

### Read files

In [3]:
# read csv files
# extract feature names available in test file when predicting
test_features = pd.read_csv('data/flights_test_raw.csv', nrows=0, index_col=0).columns.to_list()
test_features.append('arr_delay')
# read raw data
df = pd.read_csv('data/flights_raw_8900.csv', sep=',', index_col=0)
df = df[test_features]
df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay
0,2018-01-02,DL,DL,DL,1124,DL,N308DE,1124,10397,ATL,...,14524,RIC,"Richmond, VA",1725,1855,N,90.0,1.0,481.0,-15.0
1,2018-01-02,DL,DL,DL,1124,DL,N308DE,1124,14524,RIC,...,10397,ATL,"Atlanta, GA",1930,2132,N,122.0,1.0,481.0,-16.0
2,2018-01-02,DL,DL,DL,1125,DL,N342NB,1125,11057,CLT,...,10397,ATL,"Atlanta, GA",1905,2029,N,84.0,1.0,226.0,-1.0
3,2018-01-02,DL,DL,DL,1126,DL,N988DL,1126,10693,BNA,...,11433,DTW,"Detroit, MI",1843,2125,N,102.0,1.0,456.0,32.0
4,2018-01-02,DL,DL,DL,1127,DL,N951DL,1127,10397,ATL,...,12339,IND,"Indianapolis, IN",911,1042,N,91.0,1.0,432.0,-7.0


### Data Cleaning

In [4]:
df.dtypes

fl_date                object
mkt_unique_carrier     object
branded_code_share     object
mkt_carrier            object
mkt_carrier_fl_num      int64
op_unique_carrier      object
tail_num               object
op_carrier_fl_num       int64
origin_airport_id       int64
origin                 object
origin_city_name       object
dest_airport_id         int64
dest                   object
dest_city_name         object
crs_dep_time            int64
crs_arr_time            int64
dup                    object
crs_elapsed_time      float64
flights               float64
distance              float64
arr_delay             float64
dtype: object

In [5]:
# reset dtypes for categorical and time columns that contains int or float as 'str'
cat_col_names = ['mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id', 'crs_dep_time', 'crs_arr_time']
df[cat_col_names] = df[cat_col_names].astype('str')

# check dtypes again
df.dtypes

fl_date                object
mkt_unique_carrier     object
branded_code_share     object
mkt_carrier            object
mkt_carrier_fl_num     object
op_unique_carrier      object
tail_num               object
op_carrier_fl_num      object
origin_airport_id      object
origin                 object
origin_city_name       object
dest_airport_id        object
dest                   object
dest_city_name         object
crs_dep_time           object
crs_arr_time           object
dup                    object
crs_elapsed_time      float64
flights               float64
distance              float64
arr_delay             float64
dtype: object

In [6]:
# drop duplicates if there's one
df.drop_duplicates(inplace=True)

In [7]:
# check missing values information
missing_describe(df)

,dtype,missing_count,missing_percent
arr_delay,float64,206,0.001376
tail_num,object,33,0.000220
dest_airport_id,object,0,0.000000
distance,float64,0,0.000000
flights,float64,0,0.000000
crs_elapsed_time,float64,0,0.000000
dup,object,0,0.000000
crs_arr_time,object,0,0.000000
crs_dep_time,object,0,0.000000
dest_city_name,object,0,0.000000


In [8]:
# missing values in those columns are only really small portion 
# drop missing values
df.dropna(inplace=True)
df.isnull().sum().sum()

0

### Feature Engineering

In [9]:
df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay
0,2018-01-02,DL,DL,DL,1124,DL,N308DE,1124,10397,ATL,...,14524,RIC,"Richmond, VA",1725,1855,N,90.0,1.0,481.0,-15.0
1,2018-01-02,DL,DL,DL,1124,DL,N308DE,1124,14524,RIC,...,10397,ATL,"Atlanta, GA",1930,2132,N,122.0,1.0,481.0,-16.0
2,2018-01-02,DL,DL,DL,1125,DL,N342NB,1125,11057,CLT,...,10397,ATL,"Atlanta, GA",1905,2029,N,84.0,1.0,226.0,-1.0
3,2018-01-02,DL,DL,DL,1126,DL,N988DL,1126,10693,BNA,...,11433,DTW,"Detroit, MI",1843,2125,N,102.0,1.0,456.0,32.0
4,2018-01-02,DL,DL,DL,1127,DL,N951DL,1127,10397,ATL,...,12339,IND,"Indianapolis, IN",911,1042,N,91.0,1.0,432.0,-7.0


In [10]:
# check if come columns are same or similar
print('Number of rows that have different "mkt_unique_carrier" and "mkt_carrier": ', (df['mkt_unique_carrier'] != df['mkt_carrier']).sum())
print('Number of rows that have different "mkt_unique_carrier" and "branded_code_share": ', (df['mkt_unique_carrier'] != df['branded_code_share']).sum())
print('Number of rows that have different "op_unique_carrier" and "branded_code_share": ', (df['op_unique_carrier'] != df['branded_code_share']).sum())

Number of rows that have different "mkt_unique_carrier" and "mkt_carrier":  0
Number of rows that have different "mkt_unique_carrier" and "branded_code_share":  2608
Number of rows that have different "op_unique_carrier" and "branded_code_share":  2608


In [11]:
# check rows with different 'mkt_carrier', 'op_unique_carrier', 'branded_code_share'
df[df['mkt_carrier'] != df['op_unique_carrier']][['mkt_carrier', 'op_unique_carrier', 'branded_code_share']]

,mkt_carrier,op_unique_carrier,branded_code_share
57,DL,OO,DL_CODESHARE
58,DL,OO,DL_CODESHARE
59,DL,OO,DL_CODESHARE
60,DL,OO,DL_CODESHARE
61,DL,OO,DL_CODESHARE
...,...,...,...
7095,AA,YX,AA_CODESHARE
7096,AA,YX,AA_CODESHARE
7097,AA,YX,AA_CODESHARE
7098,AA,YX,AA_CODESHARE


In [12]:
# check how many rows have a mkt_carrier_fl_num different from op_carrier_fl_num
print('Number of rows that have different "mkt_carrier_fl_num" and "op_carrier_fl_num": ', (df['mkt_carrier_fl_num'] != df['op_carrier_fl_num']).sum())

Number of rows that have different "mkt_carrier_fl_num" and "op_carrier_fl_num":  3


In [13]:
# check what's in 'flights' column
df['flights'].value_counts()

1.0    6924
Name: flights, dtype: int64

In [14]:
### drop columns containing repeated information
# keep mkt_unique_carrier, drop mkt_carrier
# drop branded_code_share, since wherever 'mkt_carrier' and 'op_unique_carrier' are different, it's a shared code
# keep the 'op_carrier_fl_num', drop 'mkt_carrier_fl_num'
# keep 'origin', drop 'origin_airport_id'
# keep 'dest', drop 'dest_airport_id'
# drop 'flights'
df.drop(columns=['mkt_carrier', 'branded_code_share', 'mkt_carrier_fl_num', 'origin_airport_id', 'dest_airport_id', 'flights'], inplace=True)

# split city and country in 'origin_city_name' and 'dest_city_name' columns into 2 columns
# and drop original two columns
df[['origin_city', 'origin_state']] = df['origin_city_name'].str.split(',', expand=True, n=2)
df[['dest_city', 'dest_state']] = df['dest_city_name'].str.split(',', expand=True, n=2)
df.drop(columns=['origin_city_name', 'dest_city_name'], inplace=True)

In [18]:
# create a column of whether it's shared code
df['share_code'] = (df['mkt_unique_carrier']!=df['op_unique_carrier']).astype('int')

In [19]:
df.head()

,fl_date,mkt_unique_carrier,op_unique_carrier,tail_num,op_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,distance,arr_delay,origin_city,origin_state,dest_city,dest_state,share_code
0,2018-01-02,DL,DL,N308DE,1124,ATL,RIC,1725,1855,N,90.0,481.0,-15.0,Atlanta,GA,Richmond,VA,0
1,2018-01-02,DL,DL,N308DE,1124,RIC,ATL,1930,2132,N,122.0,481.0,-16.0,Richmond,VA,Atlanta,GA,0
2,2018-01-02,DL,DL,N342NB,1125,CLT,ATL,1905,2029,N,84.0,226.0,-1.0,Charlotte,NC,Atlanta,GA,0
3,2018-01-02,DL,DL,N988DL,1126,BNA,DTW,1843,2125,N,102.0,456.0,32.0,Nashville,TN,Detroit,MI,0
4,2018-01-02,DL,DL,N951DL,1127,ATL,IND,911,1042,N,91.0,432.0,-7.0,Atlanta,GA,Indianapolis,IN,0


In [ ]:
# pd.to_datetime(df['crs_arr_time'].astype(str).str.zfill(4), format='%H%M', errors='coerce').dt.time